In [1]:
import numpy as np
import pandas as pd

In [ ]:
revenue = consumer_sector_wise_connection_fee + consumer_sector_wise_fixed_costs + consumer_sector_wise_metering_costs + consumer_sector_wise_total_sales

## Revenue calculation

In [143]:
### constants -- taken from FATE excel 2. Control table rows 139-150
# connection fee (NGN/connection)
connection_fee = {"residential": 5000, "community": 4000, "commercial": 6000} 
# fixed costs (NGN/year/user)
fixed_costs = {"residential": 120, "community": 110, "commercial": 130} 
# metering costs (NGN/year/user)
metering_costs = {"residential": 160, "community": 150, "commercial": 170} 
# tariff (NGN/kWh) -- might actually be calculated later with goal seek function
tariffs = {"residential": 64, "community": 64, "commercial": 64} 

In [154]:
# nr consumers and energy demand (taken from tool results)
nr_consumers = {"residential": 24, "community": 18, "commercial": 7}
expected_consumer_increase = 0.02 # FATE 4. Indexation & Financing T34
nr_consumers_lifetime = {"residential": [nr_consumers["residential"]], 
                         "community": [nr_consumers["community"]], 
                         "commercial": [nr_consumers["commercial"]]}

### CAREFUL! in FATE inputs this is actually required as a daily demand, not yearly. It is later multiplied by 365
total_demand = {"residential": 62*365, "community": 105*365, "commercial": 21*365}
expected_demand_increase = 0.09 # FATE 4. Indexation & Financing
total_demand_lifetime = {"residential": [total_demand["residential"]], 
                         "community": [total_demand["community"]], 
                         "commercial": [total_demand["commercial"]]}


for group, nr in nr_consumers.items():
    demand = total_demand[group]
    for year in range(2023, 2048):
        nr = nr*(1+expected_consumer_increase)
        nr_consumers_lifetime[group].append(nr)
        demand = demand*(1+expected_demand_increase)
        total_demand_lifetime[group].append(demand)
    total_demand_lifetime[group] = np.array(total_demand_lifetime[group])
        
for group, nr_list in nr_consumers_lifetime.items():
    nr_consumers_lifetime[group] = np.array([round(nr, 0) for nr in nr_list])

### Tariffs

In [145]:
expected_tariff_increase = 0.07 # FATE 4. Indexation & Financing T34
tariffs_lifetime = {"residential": [tariffs["residential"]], 
                         "community": [tariffs["community"]], 
                         "commercial": [tariffs["commercial"]]}

for group, tariff in tariffs.items():
    for year in range(2023, 2048):
        tariff = tariff*(1+expected_tariff_increase)
        tariffs_lifetime[group].append(tariff)

    tariffs_lifetime[group] = np.array(tariffs_lifetime[group])

### Total kWh sales

In [155]:
tariffs_lifetime

{'residential': array([ 64.        ,  68.48      ,  73.2736    ,  78.402752  ,
         83.89094464,  89.76331076,  96.04674252, 102.77001449,
        109.96391551, 117.66138959, 125.89768687, 134.71052495,
        144.14026169, 154.23008001, 165.02618561, 176.57801861,
        188.93847991, 202.1641735 , 216.31566565, 231.45776224,
        247.6598056 , 264.99599199, 283.54571143, 303.39391123,
        324.63148502, 347.35568897]),
 'community': array([ 64.        ,  68.48      ,  73.2736    ,  78.402752  ,
         83.89094464,  89.76331076,  96.04674252, 102.77001449,
        109.96391551, 117.66138959, 125.89768687, 134.71052495,
        144.14026169, 154.23008001, 165.02618561, 176.57801861,
        188.93847991, 202.1641735 , 216.31566565, 231.45776224,
        247.6598056 , 264.99599199, 283.54571143, 303.39391123,
        324.63148502, 347.35568897]),
 'commercial': array([ 64.        ,  68.48      ,  73.2736    ,  78.402752  ,
         83.89094464,  89.76331076,  96.04674252, 

In [156]:
total_demand_lifetime

{'residential': array([ 22630.        ,  24666.7       ,  26886.703     ,  29306.50627   ,
         31944.0918343 ,  34819.06009939,  37952.77550833,  41368.52530408,
         45091.69258145,  49149.94491378,  53573.43995602,  58395.04955206,
         63650.60401175,  69379.1583728 ,  75623.28262636,  82429.37806273,
         89848.02208837,  97934.34407633, 106748.4350432 , 116355.79419709,
        126827.81567482, 138242.31908556, 150684.12780326, 164245.69930555,
        179027.81224305, 195140.31534492]),
 'community': array([ 38325.        ,  41774.25      ,  45533.9325    ,  49631.986425  ,
         54098.86520325,  58967.76307154,  64274.86174798,  70059.5993053 ,
         76364.96324278,  83237.80993463,  90729.21282874,  98894.84198333,
        107795.37776183, 117496.96176039, 128071.68831883, 139598.14026752,
        152161.9728916 , 165856.55045185, 180783.63999251, 197054.16759184,
        214789.0426751 , 234120.05651586, 255190.86160229, 278158.0391465 ,
        303192.2

In [164]:
consumer_sector_wise_total_sales = {}
for group in tariffs_lifetime:
    consumer_sector_wise_total_sales[group] = tariffs_lifetime[group]*total_demand_lifetime[group]
    consumer_sector_wise_total_sales[group][0] = 0
    
consumer_sector_wise_total_sales

{'residential': array([       0.        ,  1689175.616     ,  1970085.5209408 ,
         2297710.74307326,  2679820.03964634,  3125474.11223953,
         3645240.45710496,  4251443.94512151,  4958459.07319522,
         5783050.81706759,  6744772.16794593,  7866427.77947534,
         9174614.71920209, 10700353.1470054 , 12479821.8753524 ,
        14555216.2532235 , 16975748.71613457, 19798815.72762775,
        23091358.78313226, 26931451.74876715, 31410152.17458714,
        36633660.48122098, 42725838.21924803, 49831145.11510899,
        58118064.54775162, 67783098.68204273]),
 'community': array([0.00000000e+00, 2.86070064e+06, 3.33643516e+06, 3.89128432e+06,
        4.53840491e+06, 5.29314164e+06, 6.17339110e+06, 7.20002604e+06,
        8.39739037e+06, 9.79387638e+06, 1.14225980e+07, 1.33221761e+07,
        1.55376540e+07, 1.81215658e+07, 2.11351822e+07, 2.46499630e+07,
        2.87492519e+07, 3.35302524e+07, 3.91063334e+07, 4.56097167e+07,
        5.31946126e+07, 6.20408766e+07, 7.23

### Connection fees per year 

In [86]:
# calculate number of new consumers
new_consumers_lifetime = {group: np.diff(nr_consumers_lifetime[group]) for group in nr_consumers_lifetime}

# add initial consumer number to year 0
for group in new_consumers_lifetime:
    new_consumers_lifetime[group] = np.insert(new_consumers_lifetime[group], 0, nr_consumers[group]) 

In [87]:
nr_consumers_lifetime["residential"]

array([24., 24., 25., 25., 26., 26., 27., 28., 28., 29., 29., 30., 30.,
       31., 32., 32., 33., 34., 34., 35., 36., 36., 37., 38., 39., 39.])

In [88]:
new_consumers_lifetime["residential"]

array([24.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,
        1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.])

In [90]:
# general cost indexation
cost_index = 0.1 #inflation rate (taken as constant for Nigeria - FATE 4. Indexation & Financing T19)
indexation = 1
general_cost_indexation = [indexation]

for year in range(2023, 2048):
    indexation = indexation*(1+cost_index)
    general_cost_indexation.append(indexation)
    
general_cost_indexation = np.array(general_cost_indexation)

In [92]:
### this dict should contain the same data as the table in Data Mini Grid rows 33-36
# connection fee for each new consumer (all in year 0 and growing according to pop. growth)
consumer_sector_wise_connection_fee = {group: new_consumers_lifetime[group]*connection_fee[group]*general_cost_indexation 
                                       for group in nr_consumers_lifetime}

In [93]:
consumer_sector_wise_connection_fee

{'residential': array([120000.        ,      0.        ,   6050.        ,      0.        ,
          7320.5       ,      0.        ,   8857.805     ,   9743.5855    ,
             0.        ,  11789.738455  ,      0.        ,  14265.58353055,
             0.        ,  17261.35607197,  18987.49167916,      0.        ,
         22974.86493179,  25272.35142496,      0.        ,  30579.54522421,
         33637.49974663,      0.        ,  40701.37469342,  44771.51216276,
         49248.66337904,      0.        ]),
 'community': array([72000.        ,     0.        ,  4840.        ,     0.        ,
            0.        ,  6442.04      ,     0.        ,  7794.8684    ,
            0.        ,  9431.790764  ,     0.        ,     0.        ,
        12553.71350688,     0.        , 15189.99334333,     0.        ,
        18379.89194543,     0.        , 22239.66925397,     0.        ,
        26909.9997973 ,     0.        , 32561.09975474,     0.        ,
        39398.93070323, 43338.82377355])

### Fixed costs

In [112]:
# fixed charges indexation 
cost_index = 0.065 # FATE 4. Indexation & Financing T19 - it is possible to set these as different for each group, double check our approach
fixed_costs_lifetime = {"residential": [fixed_costs["residential"]], 
                         "community": [fixed_costs["community"]], 
                         "commercial": [fixed_costs["commercial"]]}

for group, costs in fixed_costs.items():
    for year in range(2023, 2048):
        costs = costs*(1+cost_index)
        fixed_costs_lifetime[group].append(costs)
    # set the starting year costs to 0
    fixed_costs_lifetime[group][0] = 0
    fixed_costs_lifetime[group] = np.array(fixed_costs_lifetime[group])

In [108]:
### this dict should contain the same data as the table in Data Mini Grid rows 39-41
# the 12 is because the costs are apparently monthly, even though in the input sheet they are described as yearly
# (yearly in control table, monthly in indexation and financing)
consumer_sector_wise_fixed_costs = {group: (nr_consumers[group]+new_consumers_lifetime[group])*12*fixed_costs_lifetime[group] 
                                       for group in nr_consumers_lifetime} 

In [110]:
consumer_sector_wise_fixed_costs

{'residential': array([     0.        ,  36806.4       ,  40832.1       ,  41746.73904   ,
         46312.7886225 ,  47350.19508764,  52529.12267536,  55943.51564925,
         57196.6503998 ,  63452.53403727,  64873.87079971,  71969.45041843,
         73581.5661078 ,  81629.54990084,  86935.4706444 ,  88882.82518683,
         98604.38419164, 105013.6691641 , 107365.97535337, 119109.12890765,
        126851.22228664, 129692.68966586, 143877.82759807, 153229.88639194,
        163189.82900742, 166845.28117719]),
 'community': array([     0.        ,  25304.4       ,  28446.363     ,  28700.88309   ,
         30566.44049085,  34361.77351846,  34669.22096573,  38973.98256898,
         39322.69714986,  44205.2653793 ,  44600.7861748 ,  47499.83727616,
         53397.73373795,  53875.50293455,  60565.04454893,  61106.94231595,
         68694.38765351,  69309.0216483 ,  77914.8918363 ,  78612.02507905,
         88373.01819303,  89163.72414528, 100234.88655999, 101131.72501868,
        113688.9

### Metering costs

In [113]:
# metering costs indexation 
cost_index = 0.065 # FATE 4. Indexation & Financing T26 - it is possible to set these as different for each group, double check our approach
metering_costs_lifetime = {"residential": [metering_costs["residential"]], 
                         "community": [metering_costs["community"]], 
                         "commercial": [metering_costs["commercial"]]}

for group, costs in metering_costs.items():
    for year in range(2023, 2048):
        costs = costs*(1+cost_index)
        metering_costs_lifetime[group].append(costs)
    # set the starting year costs to 0
    metering_costs_lifetime[group][0] = 0
    metering_costs_lifetime[group] = np.array(metering_costs_lifetime[group])

In [115]:
### this dict should contain the same data as the table in Data Mini Grid rows 45-48
# the 12 is because the costs are apparently monthly, even though in the input sheet they are described as yearly
# (yearly in control table, monthly in indexation and financing)
consumer_sector_wise_metering_costs = {group: (nr_consumers[group]+new_consumers_lifetime[group])*12*metering_costs_lifetime[group] 
                                       for group in nr_consumers_lifetime} 

In [116]:
consumer_sector_wise_metering_costs

{'residential': array([     0.        ,  49075.2       ,  54442.8       ,  55662.31872   ,
         61750.38483   ,  63133.59345019,  70038.83023381,  74591.354199  ,
         76262.20053306,  84603.37871637,  86498.49439961,  95959.26722457,
         98108.7548104 , 108839.39986779, 115913.96085919, 118510.43358244,
        131472.51225552, 140018.22555213, 143154.6338045 , 158812.17187686,
        169134.96304886, 172923.58622115, 191837.10346409, 204306.51518926,
        217586.43867656, 222460.37490291]),
 'community': array([     0.        ,  34506.        ,  38790.495     ,  39137.56785   ,
         41681.50976025,  46856.96388881,  47276.21040782,  53146.33986679,
         53621.85974981,  60279.90733541,  60819.25387473,  64772.50537658,
         72815.09146084,  73466.59491076,  82588.69711218,  83327.64861265,
         93674.16498206,  94512.30224769, 106247.57977677, 107198.21601688,
        120508.66117231, 121586.89656175, 136683.93621816, 137906.89775275,
        155030.3

In [183]:
consumer_sector_wise_total_sales

{'residential': array([       0.        ,  1689175.616     ,  1970085.5209408 ,
         2297710.74307326,  2679820.03964634,  3125474.11223953,
         3645240.45710496,  4251443.94512151,  4958459.07319522,
         5783050.81706759,  6744772.16794593,  7866427.77947534,
         9174614.71920209, 10700353.1470054 , 12479821.8753524 ,
        14555216.2532235 , 16975748.71613457, 19798815.72762775,
        23091358.78313226, 26931451.74876715, 31410152.17458714,
        36633660.48122098, 42725838.21924803, 49831145.11510899,
        58118064.54775162, 67783098.68204273]),
 'community': array([0.00000000e+00, 2.86070064e+06, 3.33643516e+06, 3.89128432e+06,
        4.53840491e+06, 5.29314164e+06, 6.17339110e+06, 7.20002604e+06,
        8.39739037e+06, 9.79387638e+06, 1.14225980e+07, 1.33221761e+07,
        1.55376540e+07, 1.81215658e+07, 2.11351822e+07, 2.46499630e+07,
        2.87492519e+07, 3.35302524e+07, 3.91063334e+07, 4.56097167e+07,
        5.31946126e+07, 6.20408766e+07, 7.23

In [177]:
revenue = {}
for group in consumer_sector_wise_connection_fee:
    revenue[group] = consumer_sector_wise_connection_fee[group] + consumer_sector_wise_fixed_costs[group] + consumer_sector_wise_metering_costs[group] + consumer_sector_wise_total_sales[group]

In [195]:
total_revenue = np.sum(list(revenue.values()), axis=0)

In [196]:
total_revenue

array([2.34000000e+05, 5.29454638e+06, 6.17579194e+06, 7.16293977e+06,
       8.35937238e+06, 9.70991483e+06, 1.13034513e+07, 1.31708293e+07,
       1.53034368e+07, 1.78538819e+07, 2.07783062e+07, 2.41978843e+07,
       2.81839096e+07, 3.28384452e+07, 3.82830760e+07, 4.45518101e+07,
       5.20151045e+07, 6.05427538e+07, 7.05541693e+07, 8.22407992e+07,
       9.58978985e+07, 1.11748558e+08, 1.30302378e+08, 1.51849499e+08,
       1.77081346e+08, 2.06376336e+08])